In [ ]:
import pandas as pd
import numpy as np
import feature_extraction as fe
from tqdm import tqdm
import pickle
import os
import ast
from sklearn.model_selection import train_test_split

ImportError: cannot import name 'Tokenizer' from 'tweetnlp' (/home/tammy.kojima/miniconda3/envs/meu_ambiente/lib/python3.9/site-packages/tweetnlp/__init__.py)

In [ ]:
df_gpt = pd.read_csv('df_pronto/df_gpt.csv')
df_human = pd.read_csv('df_pronto/df_human.csv')
#df_gpt = df_gpt.iloc[:10000]
df_human = df_human.iloc[:100000]
df_test = pd.concat([df_gpt, df_human], ignore_index=True)
df_test

,text,origin,model
0,"Our rules are the most robust, the president c...",AI,chat gpt
1,...associating outsiders with danger and extre...,AI,chat gpt
2,...alarms were raised in the initial stages of...,AI,chat gpt
3,[The opposition] utilizes every tool to questi...,AI,chat gpt
4,[The governor's] fixation with health protocol...,AI,chat gpt
...,...,...,...
199995,modi took revenge you bcos you are advani team...,human,NaN
199996,really doubt any organisation allowed function...,human,NaN
199997,⃣ already possession this capabilities but thi...,human,NaN
199998,although nasa doesnt have space suits for wome...,human,NaN


In [16]:
# 2. Selecionar a coluna de textos
# (substitua 'texto' pelo nome real da coluna com o conteúdo)
textos = df_test["text"].fillna("").tolist()

In [17]:
# Lista para armazenar resultados
lexical_features = []

for texto in tqdm(textos):
    # Tokeniza o texto usando Twokenizer
    words = fe.preprocess_text(texto)  # já retorna tokens em lowercase
    
    # Extrai features léxicas
    features = fe.extract_lexical_features(texto, words)
    lexical_features.append(features)

100%|██████████| 200000/200000 [37:08<00:00, 89.73it/s]  


In [18]:
df_lexical = pd.DataFrame(lexical_features)
df_lexical.head()

,lexical_type_token_ratio,lexical_word_count,lexical_unique_words,lexical_avg_word_length,lexical_word_length_variance,lexical_stopword_ratio
0,0.882353,17,15,5.294118,6.442907,0.411765
1,1.000000,18,18,6.000000,9.000000,0.277778
2,0.937500,32,30,4.968750,4.655273,0.437500
3,0.727273,22,16,5.409091,8.787190,0.500000
4,0.923077,13,12,6.153846,7.207101,0.384615


In [19]:
df_test = pd.concat([df_test, df_lexical], axis=1)
df_test.head()

,text,origin,model,lexical_type_token_ratio,lexical_word_count,lexical_unique_words,lexical_avg_word_length,lexical_word_length_variance,lexical_stopword_ratio
0,"Our rules are the most robust, the president c...",AI,chat gpt,0.882353,17,15,5.294118,6.442907,0.411765
1,...associating outsiders with danger and extre...,AI,chat gpt,1.000000,18,18,6.000000,9.000000,0.277778
2,...alarms were raised in the initial stages of...,AI,chat gpt,0.937500,32,30,4.968750,4.655273,0.437500
3,[The opposition] utilizes every tool to questi...,AI,chat gpt,0.727273,22,16,5.409091,8.787190,0.500000
4,[The governor's] fixation with health protocol...,AI,chat gpt,0.923077,13,12,6.153846,7.207101,0.384615


In [20]:
# Tokeniza todos os textos primeiro
processed_texts = [fe.preprocess_text(texto) for texto in tqdm(textos, desc="Tokenizando textos")]
stylistic_features = [
    fe.extract_stylistic_features(textos[i], len(processed_texts[i]))
    for i in tqdm(range(len(textos)), desc="Extraindo features estilísticas")
]
df_syntactic = pd.DataFrame(stylistic_features)
df_syntactic.head()

Extraindo features estilísticas: 100%|██████████| 200000/200000 [00:09<00:00, 21115.84it/s]


,stylistic_random_uppercase,stylistic_repeated_chars,stylistic_repeated_words,stylistic_exclamation_density,stylistic_question_density,stylistic_ellipsis_count,stylistic_emoji_density,stylistic_emoticon_density,stylistic_capitalization_inconsistency
0,0,0,0,0.0,0.0,0,0.0,0.0,0.000000
1,0,1,0,0.0,0.0,1,0.0,0.0,0.055556
2,0,1,0,0.0,0.0,1,0.0,0.0,0.000000
3,0,0,0,0.0,0.0,0,0.0,0.0,0.000000
4,0,0,0,0.0,0.0,0,0.0,0.0,0.000000


In [25]:
df_test = pd.concat([df_test, df_syntactic], axis=1)
df_test.head()

,text,origin,model,lexical_type_token_ratio,lexical_word_count,lexical_unique_words,lexical_avg_word_length,lexical_word_length_variance,lexical_stopword_ratio,stylistic_random_uppercase,stylistic_repeated_chars,stylistic_repeated_words,stylistic_exclamation_density,stylistic_question_density,stylistic_ellipsis_count,stylistic_emoji_density,stylistic_emoticon_density,stylistic_capitalization_inconsistency
0,"Our rules are the most robust, the president c...",AI,chat gpt,0.882353,17,15,5.294118,6.442907,0.411765,0,0,0,0.0,0.0,0,0.0,0.0,0.000000
1,...associating outsiders with danger and extre...,AI,chat gpt,1.000000,18,18,6.000000,9.000000,0.277778,0,1,0,0.0,0.0,1,0.0,0.0,0.055556
2,...alarms were raised in the initial stages of...,AI,chat gpt,0.937500,32,30,4.968750,4.655273,0.437500,0,1,0,0.0,0.0,1,0.0,0.0,0.000000
3,[The opposition] utilizes every tool to questi...,AI,chat gpt,0.727273,22,16,5.409091,8.787190,0.500000,0,0,0,0.0,0.0,0,0.0,0.0,0.000000
4,[The governor's] fixation with health protocol...,AI,chat gpt,0.923077,13,12,6.153846,7.207101,0.384615,0,0,0,0.0,0.0,0,0.0,0.0,0.000000


In [26]:
df_test.to_csv('df_pronto/df_2pronto.csv', index=False)

In [ ]:
df_processed_texts = pd.DataFrame({'processed_text': processed_texts})
df_processed_texts

,processed_text
0,"[our, rules, are, the, most, robust, the, pres..."
1,"[associating, outsiders, with, danger, and, ex..."
2,"[alarms, were, raised, in, the, initial, stage..."
3,"[the, opposition, utilizes, every, tool, to, q..."
4,"[the, governor, 's, fixation, with, health, pr..."
...,...
199995,"[modi, took, revenge, you, bcos, you, are, adv..."
199996,"[really, doubt, any, organisation, allowed, fu..."
199997,"[⃣, already, possession, this, capabilities, b..."
199998,"[although, nasa, does, nt, have, space, suits,..."


In [23]:
df_processed_texts.to_csv('df_pronto/df_tokens_gpt.csv', index=False)

In [ ]:
df_test = pd.read_csv('df_pronto/df_2pronto.csv')
textos = df_test["text"].fillna("").tolist()

/tmp/ipykernel_764190/2166943379.py:1: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df_test = pd.read_csv('df_pronto/df_2pronto.csv')


In [7]:
# Tokeniza todos os textos primeiro
# FALTA FAZER ESSEEEEEEEEEE
#
#
#
df_test = pd.read_csv('df_pronto/df_2pronto.csv')
textos = df_test["text"].fillna("").tolist()
processed_texts = [fe.preprocess_text_A(texto) for texto in tqdm(textos, desc="Tokenizando textos")]
stylistic_features = [
    fe.extract_stylistic_featuresA(textos[i], len(processed_texts[i]))
    for i in tqdm(range(len(textos)), desc="Extraindo features estilísticas")
]
df_syntactic = pd.DataFrame(stylistic_features)
df_syntactic.head()

/tmp/ipykernel_764190/763372016.py:6: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df_test = pd.read_csv('df_pronto/df_2pronto.csv')


NameError: name 'fe' is not defined

In [ ]:
df_test = pd.read_csv('df_pronto/df_2pronto.csv')
df_processed_texts = pd.read_csv('df_pronto/df_tokens_gpt.csv')
processed_texts = df_processed_texts['processed_text'].apply(ast.literal_eval).tolist()
textos = df_test["text"].fillna("").tolist()

/tmp/ipykernel_702265/2159682550.py:1: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df_test = pd.read_csv('df_pronto/df_3pronto.csv')


In [ ]:
syntactic_features = [
    fe.extract_syntactic_features(processed_texts[i])
    for i in tqdm(range(len(textos)), desc="Extraindo features sintáticas")
]
df_syntactic = pd.DataFrame(syntactic_features)
df_syntactic.head()

NameError: name 'fe' is not defined

In [ ]:
structural_features = [
    fe.extract_structural_features(textos[i])
    for i in tqdm(range(len(textos)), desc="Extraindo features estruturais")
]
df_structural = pd.DataFrame(structural_features)
df_structural.head()

Extraindo Features Estruturais: 100%|██████████| 200000/200000 [07:43<00:00, 431.43it/s]


,structural_has_url,structural_has_mention,structural_has_hashtag,structural_is_retweet,structural_url_density,structural_mention_density,structural_hashtag_density,structural_extra_spaces,structural_temporal_markers
0,0,0,0,0,0.0,0.0,0.0,0.0,0.0
1,0,0,0,0,0.0,0.0,0.0,0.0,0.0
2,0,0,0,0,0.0,0.0,0.0,0.0,0.0
3,0,0,0,0,0.0,0.0,0.0,0.0,0.0
4,0,0,0,0,0.0,0.0,0.0,0.0,0.0


In [24]:
df_test = pd.concat([df_test, df_structural], axis=1)
df_test.to_csv('df_pronto/df_3pronto.csv', index=False)

In [3]:
# Exemplo de uso básico
ngrams_df = fe.extract_large_ngrams(textos, max_features=500, batch_size=20000)
ngrams_df.head()

Processando N-grams: 100%|██████████| 10/10 [12:52<00:00, 77.24s/it]


,10,2023,about,about climate,about climate change,about how,about the,across,act,action,...,the scientists,took,towards,updates,vision,waiting for,weapon,with important,with important message,you modi
0,0,0,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,0,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0,0,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0,0,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0,0,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
# Resetar os índices de ambos os DataFrames
df_test_reset = df_test.reset_index(drop=True)
ngrams_df_reset = ngrams_df.reset_index(drop=True)

# Agora concatenar
df_combined = pd.concat([df_test_reset, ngrams_df_reset], axis=1)

In [6]:
df_combined.head()

,text,origin,model,lexical_type_token_ratio,lexical_word_count,lexical_unique_words,lexical_avg_word_length,lexical_word_length_variance,lexical_stopword_ratio,syntactic_pos_tag_entropy,...,the scientists,took,towards,updates,vision,waiting for,weapon,with important,with important message,you modi
0,"Our rules are the most robust, the president c...",AI,chat gpt,0.882353,17,15,5.294118,6.442907,0.411765,2.032114,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,...associating outsiders with danger and extre...,AI,chat gpt,1.000000,18,18,6.000000,9.000000,0.277778,1.933809,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,...alarms were raised in the initial stages of...,AI,chat gpt,0.937500,32,30,4.968750,4.655273,0.437500,2.089713,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,[The opposition] utilizes every tool to questi...,AI,chat gpt,0.727273,22,16,5.409091,8.787190,0.500000,1.894287,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,[The governor's] fixation with health protocol...,AI,chat gpt,0.923077,13,12,6.153846,7.207101,0.384615,1.927392,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
# Remove a coluna 'model'
#df_combined = df_combined.drop(columns=['model'])

# Troca IA → 1 e human → 0
df_combined['origin'] = df_combined['origin'].replace({'AI': 1, 'human': 0})

df_combined.head()

/tmp/ipykernel_702265/4028979220.py:5: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_combined['origin'] = df_combined['origin'].replace({'AI': 1, 'human': 0})


,text,origin,lexical_type_token_ratio,lexical_word_count,lexical_unique_words,lexical_avg_word_length,lexical_word_length_variance,lexical_stopword_ratio,syntactic_pos_tag_entropy,syntactic_pos_bigram_entropy,...,the scientists,took,towards,updates,vision,waiting for,weapon,with important,with important message,you modi
0,"Our rules are the most robust, the president c...",1,0.882353,17,15,5.294118,6.442907,0.411765,2.032114,2.780466,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,...associating outsiders with danger and extre...,1,1.000000,18,18,6.000000,9.000000,0.277778,1.933809,2.902002,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,...alarms were raised in the initial stages of...,1,0.937500,32,30,4.968750,4.655273,0.437500,2.089713,3.198872,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,[The opposition] utilizes every tool to questi...,1,0.727273,22,16,5.409091,8.787190,0.500000,1.894287,2.375897,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,[The governor's] fixation with health protocol...,1,0.923077,13,12,6.153846,7.207101,0.384615,1.927392,2.523211,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
df_combined = df_combined.apply(lambda x: x.sparse.to_dense() if pd.api.types.is_sparse(x) else x)
df_combined.describe()

/tmp/ipykernel_702265/91803334.py:1: DeprecationWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  df_combined = df_combined.apply(lambda x: x.sparse.to_dense() if pd.api.types.is_sparse(x) else x)


,origin,lexical_type_token_ratio,lexical_word_count,lexical_unique_words,lexical_avg_word_length,lexical_word_length_variance,lexical_stopword_ratio,syntactic_pos_tag_entropy,syntactic_pos_bigram_entropy,syntactic_avg_sentence_length,...,the scientists,took,towards,updates,vision,waiting for,weapon,with important,with important message,you modi
count,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,...,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.0000,20000.000000
mean,0.500000,0.939904,20.395885,18.869735,5.145131,10.948684,0.421463,2.029532,2.716272,14.310194,...,0.007200,0.009000,0.007000,0.009100,0.008800,0.008400,0.013100,0.006650,0.0066,0.006600
std,0.500001,0.065652,9.664566,8.529665,1.318695,13.854440,0.159344,0.364268,0.579060,8.800167,...,0.086305,0.097054,0.086899,0.095486,0.104991,0.093434,0.116315,0.082499,0.0822,0.080974
min,0.000000,0.076923,1.000000,1.000000,1.000000,0.000000,0.000000,-0.000000,-0.000000,0.500000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000
25%,0.000000,0.903226,12.000000,12.000000,4.375000,4.093333,0.333333,1.848011,2.458311,8.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000
50%,0.500000,0.950000,20.000000,19.000000,4.944444,6.616071,0.437500,2.117277,2.886165,12.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000
75%,1.000000,1.000000,28.000000,26.000000,5.608696,12.290000,0.533333,2.290368,3.130860,17.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000
max,1.000000,1.000000,107.000000,76.000000,76.000000,2348.913580,1.000000,2.720884,3.912701,107.000000,...,2.000000,2.000000,2.000000,2.000000,3.000000,2.000000,2.000000,2.000000,2.0000,1.000000


In [ ]:
df_combined.to_csv("df_pronto/df_gpt_com_features.csv", index=False)